https://github.com/ustroetz/python-osrm

OSMR API documentation: http://project-osrm.org/docs/v5.24.0/api/?language=cURL#

Выжимка:

**Выбирается самый быстрый путь, а не минимальное расстояние между двумя координатами!**

**Расстояние указывается в метрах**

**Open Source OSMR API поддерживает только пути для машин. Есть попытаться указать другие опции, результат тот же.**

**route** — посчитает путь со всеми указанными координатами.

options:
    * skip_waypoints=true — не будет выводить информацию об указанных координатах. Позволяет экономить место;
    * alternatives=n — показать альтернативные пути;
    * geometries=geojson — возвращает список пар координат для построения пути;
    * overview=full — полный, simplified — не подробный, уменьшает количество пар координат, false — без геометрии;

response:
    * code — обозначает успешность запроса (200 успешно, 400 — ошибка);
    * waypoints — точки на пути, исчезнут при skip_waypoints=true;
    * routes — пути, отранжированные от самого быстрого к самому медленному.

**table** — посчитает пути между всеми парами координат, переданными в запрос.

options:
    * skip_waypoints=true — не будет выводить информацию об указанных координатах. Позволяет экономить место, но также обнулит sources и destinations;
    * sources — координаты, используемые в качестве отправной точки, вернутся в качестве waypoint объекта
    * destinations — координаты, используемые в качестве пункта назначения, вернутся в качестве waypoint объекта
    * annotations — duration (default), distance , or duration,distance — что возвращается;
    * fallback_speed — в случае, если между точкам не удается найти путь, будет посчитано кратчайшее расстояние, для которого необходимо указать скорость.

response:
    * code
    * durations — null, если путь не удалось построить;
    * distances — null, если путь не удалось построить;
    * sources
    * destinations
    * fallback_speed_cells — если был использован fallback_speed, то вернется массив, указывающий, какие пары были рассчитаны подобным образом.
    
Макет ссылки:

http://router.project-osrm.org/route/v1/driving/59.98259700478887,30.4297523923562;55.75269662241035,37.64085841204411?overview=false

In [17]:
import requests
import json
import folium

In [50]:
response = requests.get('http://router.project-osrm.org/route/v1/driving/59.98259700478887,30.4297523923562;55.75269662241035,37.64085841204411?overview=simplified&geometries=geojson')

In [51]:
route = json.loads(response.text)

In [75]:
route['routes'][0]['geometry']['coordinates']

[[60.069109, 30.413012],
 [60.465952, 29.806406],
 [60.819307, 29.56304],
 [60.842607, 30.55827],
 [60.05774, 31.52117],
 [60.065085, 31.765116],
 [59.827911, 32.066506],
 [59.782394, 32.621947],
 [59.247207, 32.897106],
 [59.303573, 33.048077],
 [59.279291, 33.278319],
 [59.169153, 33.417222],
 [59.192266, 33.708848],
 [59.113299, 33.849785],
 [58.834977, 33.923343],
 [58.784619, 34.325181],
 [58.677108, 34.388643],
 [58.853752, 34.780219],
 [58.840263, 34.996685],
 [59.268687, 35.268877],
 [59.178742, 35.507834],
 [59.260419, 35.775053],
 [59.403036, 35.823299],
 [59.427114, 35.937947],
 [59.679679, 36.011124],
 [59.72371, 36.127514],
 [59.516989, 36.386918],
 [58.022246, 37.380988],
 [57.50293, 37.517831],
 [57.343914, 37.466313],
 [56.713674, 37.587558],
 [56.373233, 37.73668],
 [56.174106, 37.63983],
 [55.751698, 37.649059]]

In [79]:
route['routes'][0]['distance']

1501412.4

In [53]:
r = route['routes'][0]['geometry']['coordinates']

In [54]:
m = folium.Map(location=[59.98259700478887,30.4297523923562],
              zoom_start=5)

In [55]:
folium.PolyLine(r,
                color='red',
                weight=15,
                opacity=0.8).add_to(m)

In [56]:
m

# Пробуем на датасете

In [21]:
import psycopg2
import numpy as np
from scipy.cluster.hierarchy import fclusterdata
from scipy.spatial.distance import pdist
from math import pi, sin, cos, atan2
import matplotlib.pyplot as plt
import pandas as pd
from itertools import combinations
from tqdm import tqdm

In [2]:
conn = psycopg2.connect(dbname='sociolinguistic', user='app', 
                        password='rfrfyl.babaloos', host='gisly.net')
cursor = conn.cursor()

In [3]:
cursor.execute(""" select place_born.longitude, place_born.latitude
from sociolinguistic_place place_born
where place_born.longitude is not null
and place_born.latitude is not null""")

In [4]:
X = np.array(cursor.fetchall())

In [5]:
cursor.close()
conn.close()

In [11]:
X[0]

array([Decimal('141.837700'), Decimal('53.165500')], dtype=object)

In [14]:
table = list(combinations(X, 2))
longitude_x = [x[0][0] for x in table]
latitude_x = [x[0][1] for x in table]
longitude_y = [x[1][0] for x in table]
latitude_y = [x[1][1] for x in table]
df = pd.DataFrame()
df['longitude_x'] = longitude_x
df['latitude_x'] = latitude_x
df['longitude_y'] = longitude_y
df['latitude_y'] = latitude_y
df['distance'] = np.nan
df['duration'] =  np.nan
df['geometry'] = np.nan

In [20]:
def calculate_dist(place1, place2):
    place1 = str(place1[0]) + ',' + str(place1[1])
    place2 = str(place2[0]) + ',' + str(place2[1])
    try:
        response = requests.get(
            'http://router.project-osrm.org/route/v1/driving/{0};{1}?overview=simplified&geometries=geojson'.format(
            place1, place2))
        response = json.loads(response.text)
        distance = response['routes'][0]['distance']
        duration = response['routes'][0]['duration']
        geometry = json.dumps(response['routes'][0]['geometry']['coordinates'])
    except Exception as e:
        distance = np.nan
        duration = np.nan
        geometry = np.nan
    return distance, duration, geometry

In [ ]:
for num, row in tqdm(df.iterrows()):
    distance, duration, geometry = calculate_dist([row['longitude_x'], row['latitude_x']],
                                                 [row['longitude_y'], row['latitude_y']])
    df.loc[num, 'distance'] = distance
    df.loc[num, 'duration'] = duration
    df.loc[num, 'geometry'] = geometry

22043it [3:03:44,  1.98it/s]